In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec,FastText
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
train_data = pd.read_csv("drive/My Drive/Sequential Labelling/dataset/train.csv", encoding="utf-8",names=['chunks'])
train_data[['Words','POS','TAG']] = train_data["chunks"].str.split(" ", 2, expand=True)
train_data = train_data.drop(columns=['chunks'])
train_data

,Words,POS,TAG
0,Confidence,NN,B-NP
1,in,IN,B-PP
2,the,DT,B-NP
3,pound,NN,I-NP
4,is,VBZ,B-VP
...,...,...,...
88184,the,DT,B-NP
88185,scorecard,NN,I-NP
88186,.,.,O
88187,NaN,NaN,NaN


In [4]:
test_data = pd.read_csv("drive/My Drive/Sequential Labelling/dataset/test.csv", encoding="utf-8",names=['chunks'])
test_data[['Words','POS','TAG']] = test_data["chunks"].str.split(" ", 2, expand=True)
test_data = test_data.drop(columns=['chunks'])
test_data

,Words,POS,TAG
0,Rockwell,NNP,B-NP
1,International,NNP,I-NP
2,Corp.,NNP,I-NP
3,'s,POS,B-NP
4,Tulsa,NNP,I-NP
...,...,...,...
49384,to,TO,B-PP
49385,Mr.,NNP,B-NP
49386,Harlow,NNP,I-NP
49387,.,.,O


In [5]:
print("Null Train:")
print(train_data.isnull().sum())
print("Null Test:")
print(test_data.isnull().sum())

Null Train:
Words    3572
POS      3572
TAG      3572
dtype: int64
Null Test:
Words    2012
POS      2012
TAG      2012
dtype: int64


In [6]:
train_data.dropna(subset=['Words','POS','TAG'], inplace=True)
train_data = train_data.reset_index()

test_data.dropna(subset=['Words','POS','TAG'], inplace=True)
test_data = test_data.reset_index()

In [35]:
# train_data['NumOfSentence'] = "Sentence"
# test_data['NumOfSentence'] = "Sentence"

In [36]:
# j = 1
# for i in range(0,len(train_data)):
#   if train_data['TAG'][i] == 'O':
#     j +=1
#     train_data['NumOfSentence'][i] += str(j+1)
#   else:
#     train_data['NumOfSentence'][i] += str(j)

In [8]:
train_data = train_data[~train_data.TAG.str.contains("O")]
train_data = train_data.reset_index()

test_data = test_data[~test_data.TAG.str.contains("O")]
test_data = test_data.reset_index()

In [10]:
# dTrain = [[str(train_data['NumOfSentence'][i])+"_"+str(train_data['Words'][i])+"_"+str(train_data['POS'][i])] for i in range(0,len(train_data))]
dTrain = [[str(train_data['Words'][i])+"_"+str(train_data['POS'][i])] for i in range(0,len(train_data))]
dTest = [[str(test_data['Words'][i])+"_"+str(test_data['POS'][i])] for i in range(0,len(test_data))]

In [11]:
# max_epochs = 5
vec_size = 50
model = FastText(min_count=1)
model.build_vocab(dTrain)  # prepare the model vocabulary
model.train(dTrain, total_examples=model.corpus_count, epochs=5)

In [12]:
len(model.wv.vocab)

11923

In [13]:
def get_vec(model,dTrain,lab):
  vector = []
  tag = []
  for i in range(0,len(dTrain)):
    vector.append(model.wv[dTrain[i]])
    tag.append(lab[i])
  return vector,tag


In [14]:
train_vec,train_tag = get_vec(model,dTrain,train_data.TAG)
test_vec,test_tag = get_vec(model,dTest,test_data.TAG)

In [17]:
print("label train : ",list(set(train_tag)))
print("label test  : ",list(set(test_tag)))

label train :  ['I-ADVP', 'I-PP', 'B-LST', 'I-SBAR', 'B-PP', 'B-ADVP', 'B-INTJ', 'I-INTJ', 'B-ADJP', 'B-NP', 'I-NP', 'B-SBAR', 'I-PRT', 'I-VP', 'I-ADJP', 'B-VP', 'B-PRT']
label test  :  ['I-ADVP', 'I-PP', 'B-LST', 'I-SBAR', 'B-PP', 'B-ADVP', 'I-LST', 'B-INTJ', 'B-ADJP', 'B-NP', 'I-NP', 'B-SBAR', 'I-VP', 'I-ADJP', 'B-VP', 'B-PRT']


In [311]:
np.shape(train_label)

(73315,)

In [18]:
X_train,X_valid,y_train,y_valid = train_test_split(train_vec, train_tag, test_size=0.2, shuffle=True)

In [19]:
X_train = np.reshape(X_train,(len(X_train),100))
X_valid = np.reshape(X_valid,(len(X_valid),100))
X_test = np.reshape(test_vec,(len(test_vec),100))

In [20]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
training1 = logreg.predict(X_train)
print('Training accuracy (Logistic Regression) : %.2f' % (accuracy_score(training1, y_train)*100),"%")
#
y_pred_valid1 = logreg.predict(X_valid)
print('Validation accuracy (Logistic Regression) : %.2f' % (accuracy_score(y_valid, y_pred_valid1)*100),"%")
# 
y_pred_test1 = logreg.predict(X_test)
print('Test accuracy (Logistic Regression) : %.2f' % (accuracy_score(test_tag, y_pred_test1)*100),"%")

Training accuracy (Logistic Regression) : 64.70 %
Validation accuracy (Logistic Regression) : 64.05 %
Test accuracy (Logistic Regression) : 64.84 %


In [22]:
RF = RandomForestClassifier(n_estimators=100,random_state=42)
RF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [34]:
training_2 = RF.predict(X_train)
print('Training accuracy (Random Forest) : %.2f' % (accuracy_score(training_2, y_train)*100),"%")
#
y_pred_valid2 = RF.predict(X_valid)
print('Validation accuracy (Random Forest) : %.2f' % (accuracy_score(y_valid, y_pred_valid2)*100),"%")

y_pred_test2 = RF.predict(X_test)
print('Test accuracy (Random Forest) : %.2f' % (accuracy_score(test_tag, y_pred_test2)*100),"%")

Training accuracy (Random Forest) : 86.67 %
Validation accuracy (Random Forest) : 79.49 %
Test accuracy (Random Forest) : 78.95 %


In [32]:
print("Similarity hasil prediksi dari 2 classifier : %.2f"% (accuracy_score(y_pred_test1, y_pred_test2)*100),"%")

Similarity hasil prediksi dari 2 classifier : 70.83 %
